<img src="data/photutils_banner.svg" width=500 alt="Photutils logo" style="margin-left: 0;">

<div class="alert alert-block alert-info">
<h2 style="margin-top: 0">In this notebook, we will cover:</h2>

- PSF photometry using simulated 2D Gaussian PSFs, using:
    - BasicPSFPhotometry
    - IterativelySubtractedPSFPhotometry
    - DAOPhotPSFPhotometry
</div>

# Preliminaries

In [ ]:
# Initial imports
import numpy as np
import matplotlib.pyplot as plt

# Change some default plotting parameters
import matplotlib as mpl
mpl.rcParams['image.origin'] = 'lower'
mpl.rcParams['image.interpolation'] = 'nearest'

# Run the %matplotlib magic command to enable inline plotting
# in the current notebook.  Choose one of these:
%matplotlib inline
# %matplotlib notebook

# Point Spread Function Photometry with Photutils

The Photutils PSF photometry module provides modular tools that allow users to completely customize the photometry procedure, e.g., by using different source detection algorithms, background estimators, and PSF models. Photutils provides implementations for each subtask involved in the photometry process, however, users are still able to include their own custom implementations.

This modularity is accomplished by using an object oriented programming approach that provides a more convenient user experience.

Photutils provides three basic classes to perform PSF photometry: `BasicPSFPhotometry`, `IterativelySubtractedPSFPhotometry`, and `DAOPhotPSFPhotometry`. In this notebook, we will go through them, explaining their differences and particular uses.

# Artificial Starlist

Let's start by creating an artificial list of stars using Photutils in order to explain the PSF procedures through examples.

In [ ]:
from photutils.datasets import (make_gaussian_sources_image, make_noise_image,
                                make_random_gaussians_table)

num_sources = 150
min_flux = 500
max_flux = 5000
min_xmean = 16
max_xmean = 240
sigma_psf = 2.0

param_ranges = dict([('flux', [500, 1000]),
                     ('x_mean', [min_xmean, max_xmean]),
                     ('y_mean', [min_xmean, max_xmean]),
                     ('x_stddev', [sigma_psf, sigma_psf]),
                     ('y_stddev', [sigma_psf, sigma_psf]),
                     ('theta', [0, np.pi])])
starlist = make_random_gaussians_table(num_sources, param_ranges,
                                       seed=1234)

`starlist` is an Astropy `QTable`.

In [ ]:
type(starlist)

In [ ]:
starlist

Most source lists in Photutils are returned or passed in as `astropy` `Table` objects, so this is something to get used to.

Now let's create an image from this source list.

We'll also add Gaussian background noise with the function `make_noise_image`.

In [ ]:
shape = (256, 256)
image = (make_gaussian_sources_image(shape, starlist) +
         make_noise_image(shape, distribution='gaussian', mean=0.0, stddev=2.0, seed=1234))

Let's plot our image of simulated stars.

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.title('Simulated data')
plt.colorbar(orientation='horizontal', fraction=0.046, pad=0.04);

# The `BasicPSFPhotometry` class

As the name suggests, this is a basic class which provides the minimum tools necessary to perform photometry in crowded fields (or non crowded fields). Let's take a look into its input parameters.

`BasicPSFPhotometry` has the following mandatory parameters:

* group_maker : callable or instance of any `GroupStarsBase` subclass
* bkg_estimator : callable, instance of any `BackgroundBase` subclass, or None
* psf_model : `astropy.modeling.Fittable2DModel` instance
* fitshape : integer or length-2 array-like

And the following optional parameters:

* finder : callable or instance of any `StarFinderBase` subclasses or None
* fitter : Astropy Fitter instance
* aperture_radius : float or int
* subshape: Rectangular shape around the center of a star that will be used to define the PSF-subtraction region.

## Group Maker

`group_maker` can be instantiated using any `GroupStarBase` subclass, such as `DAOGroup` or `DBSCANGroup`, or even using a `callable` provided by the user.

[DAOGroup](https://photutils.readthedocs.io/en/stable/api/photutils.psf.DAOGroup.html#photutils.psf.DAOGroup) is a class that implements the `GROUP` algorithm proposed by Stetson and used in DAOPHOT. This class takes one parameter, namely:

* `crit_separation` : Distance, in units of pixels, such that any two stars separated by less than this distance will be placed in the same group.

`crit_separation` plays a crucial role in deciding whether or not a given star belongs to some group of stars. Usually, `crit_separation` is set to be a positive real number multiplied by the FWHM of the PSF.

[DBSCANGroup](https://photutils.readthedocs.io/en/stable/api/photutils.psf.DBSCANGroup.html#photutils.psf.DBSCANGroup) is a generalized case of `DAOGroup` (it is a wrapper around the `sklearn.cluster.DBSCAN` class). Its usage is very similar to `DAOGroup`.

Grouping examples are available in the [Grouping Algorithms](https://photutils.readthedocs.io/en/stable/grouping.html) documentation.

Let's instantiate a `group_maker` from `DAOGroup`.

In [ ]:
from astropy.stats import gaussian_sigma_to_fwhm
from photutils import psf

daogroup = psf.DAOGroup(crit_separation=2.0 * sigma_psf * gaussian_sigma_to_fwhm)

Now, the `daogroup` object is ready to be passed to `BasicPSFPhotometry`.

## Background Estimation

Photutils provides several classes to perform both scalar background estimation, i.e., when the background is flat and does not vary strongly across the image, and spatial varying background estimation, i.e., when there exists a gradient field associated with the background. Please refer to to the Photutils [background estimation](https://photutils.readthedocs.io/en/stable/background.html) documentation for examples.

In this notebook, we will use the class `MMMBackground` that estimates a scalar background. This class is based on the background estimator used in `DAOPHOT`.

`MMMBackground` accepts a `SigmaClip` object to perform sigma clipping on the image before performing background estimation. For this example, we will  use a `MMMBackground` object with the default values.

In [ ]:
from photutils.background import MMMBackground

mmm_bkg = MMMBackground()

In [ ]:
mmm_bkg.sigma_clip.sigma

In [ ]:
mmm_bkg.sigma_clip.maxiters

## PSF Models

The ``psf_model`` parameter represents an analytical function with unknown parameters (e.g., centroid and flux) that describes the underlying point spread function. ``psf_model`` is usually a subclass of `astropy.modeling.Fittable2DModel`. In this notebook, we will use `IntegratedGaussianPRF` as our underlying PSF model.

Note that the underlying PSF model must have attributes with the names ``x_0``, ``y_0``, and ``flux``, which describe the center peak position and total flux, respectively.

In [ ]:
from photutils.psf import IntegratedGaussianPRF

gaussian_psf = IntegratedGaussianPRF(sigma=2.0)

## Finder

Finder is an optional parameter.  If it is `None`, then the user should provide a table with the center positions of each star when calling the `BasicPSFPhotometry` object.
Later, we will see examples cases when `finder` is `None` and when it is not.

The finder parameter is used to perform source detection. It can be any subclass of `StarFinderBase`, such as `DAOStarFinder` or `IRAFStarFinder`, which implement a DAOPHOT-like or IRAF-like source detection algorithms, respectively. The user can also define their own source detection algorithm as long as the input/output formats are compatible with `StarFinderBase`.

`DAOStarFinder`, for instance, receives the following mandatory parameters:
    
* `threshold` : The absolute image value above which to select sources.

* `fwhm` : The full-width half-maximum (FWHM) of the major axis of the Gaussian kernel in units of pixels.

Now, let's instantiate our `DAOStarFinder` object.

In [ ]:
from photutils.background import StdBackgroundRMS
from photutils.detection import DAOStarFinder

bkg_rms = StdBackgroundRMS()
threshold = 3.5 * bkg_rms(image)
fwhm = sigma_psf * gaussian_sigma_to_fwhm
daofinder = DAOStarFinder(threshold=threshold, fwhm=fwhm)

Note that we choose the `threshold` to be a multiple of the background noise level and we assumed the `fwhm` to be known from our list of stars.

More details about source detection can be found on the `photutils.detection` narrative docs: https://photutils.readthedocs.io/en/stable/detection.html

## Fitter

Fitter should be an instance of a fitter implemented in `astropy.modeling.fitting`. Since the PSF model is almost always nonlinear, the fitter should be able to handle nonlinear optimization problems. In this notebook, we will use  `LevMarLSQFitter`, which combines the Levenberg-Marquardt optimization algorithm with the least-squares statistic.

See at [Astropy Models and Fitting](https://docs.astropy.org/en/stable/modeling/index.html) for more details on fitting.

## Fitshape and Aperture Radius

There are three parameters left: `fitshape` (required), and `aperture_radius` and `subshape` (optional).

`fitshape` corresponds to the size of the rectangular region necessary to enclose one single source. The pixels inside that region will be used in the fitting process. `fitshape` should be an odd integer or a tuple of odd integers.

The aperture radius corresponds to the radius used to compute initial guesses for the fluxes of the sources. If this value is `None`, then one FWHM will be used if it can be determined by the `psf_model`.

`subshape` is shape around the center of a star that will be used to define the PSF-subtraction region. By default, it is the same as `fitshape`.

In [ ]:
fitshape = 11

## Example with unknown positions and unknown fluxes

Now we are ready to take a look at an actual example. Let's first create our `BasicPSFPhotometry` object putting together the pieces that we defined along the way.

In [ ]:
from photutils.psf import BasicPSFPhotometry

basic_photometry = BasicPSFPhotometry(group_maker=daogroup, bkg_estimator=mmm_bkg,
                                      psf_model=gaussian_psf, fitshape=fitshape,
                                      finder=daofinder)

To actually perform photometry on our image that we defined previously, we should use `basic_photometry` as a function call:

In [ ]:
photometry_results = basic_photometry(image)
photometry_results

Let's plot the residual image along with the original image:

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 10))

im1 = ax1.imshow(image)
ax1.set_title('Simulated data')
plt.colorbar(orientation='horizontal', fraction=0.046, pad=0.04,
             ax=ax1, mappable=im1)

im2 = ax2.imshow(basic_photometry.get_residual_image())
ax2.set_title('Residual Image')
cbar = plt.colorbar(orientation='horizontal', fraction=0.046, pad=0.04,
                    ax=ax2, mappable=im2)

Looking at the residual image we observe that the photometry process was able to fit many stars but not all. This is probably due to inability of the source detection algorithm to decide the number of sources in every crowded group. Therefore, let's play with the source detection classes to see whether we can improve the photometry process.

Let's use the `IRAFStarFinder` and change some of the optional parameters. A complete description of these parameters can be seen at the `photutils.detection` API documentation: https://photutils.readthedocs.io/en/stable/api/photutils.detection.IRAFStarFinder.html#photutils.detection.IRAFStarFinder

In [ ]:
from photutils.detection import IRAFStarFinder

iraffind = IRAFStarFinder(threshold=threshold,
                          fwhm=sigma_psf*gaussian_sigma_to_fwhm,
                          minsep_fwhm=0.01, roundhi=5.0, roundlo=-5.0,
                          sharplo=0.0, sharphi=2.0)

Now let's set the `finder` attribute of our `BasicPSFPhotometry` object with `iraffind`:

In [ ]:
basic_photometry.finder = iraffind

Let's repeat the photometry process:

In [ ]:
photometry_results = basic_photometry(image)
photometry_results

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 10))

im1 = ax1.imshow(image)
ax1.set_title('Simulated data')
plt.colorbar(orientation='horizontal', fraction=0.046, pad=0.04,
             ax=ax1, mappable=im1)

im2 = ax2.imshow(basic_photometry.get_residual_image())
ax2.set_title('Residual Image')
plt.colorbar(orientation='horizontal', fraction=0.046, pad=0.04,
             ax=ax2, mappable=im2)

As we can see, the residuals are better with only four groups that were not fitted well. The reason for that is that the sources may be too close to be distinguishable by the source detection algorithm.

## Example with known positions and unknown fluxes

Let's assume that somehow we know the true positions of the stars and we only would like to perform fitting on the fluxes. Then we should use the optional argument `init_guesses` when calling the photometry object:

In [ ]:
from astropy.table import Table

positions = Table(names=['x_0', 'y_0'], data=[starlist['x_mean'], starlist['y_mean']])

First, we turn off the star finder.

In [ ]:
basic_photometry.finder = None

Next, we must fix the PSF model x and y positions during the fitting.

In [ ]:
basic_photometry.psf_model.x_0.fixed = True
basic_photometry.psf_model.y_0.fixed = True

In [ ]:
photometry_results = basic_photometry(image=image, init_guesses=positions)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 10))

im1 = ax1.imshow(image)
ax1.set_title('Simulated data')
plt.colorbar(orientation='horizontal', fraction=0.046, pad=0.04,
             ax=ax1, mappable=im1)

im2 = ax2.imshow(basic_photometry.get_residual_image())
ax2.set_title('Residual Image')
plt.colorbar(orientation='horizontal', fraction=0.046, pad=0.04,
             ax=ax2, mappable=im2)

Let's do a scatter plot between ground-truth fluxes and estimated fluxes:

In [ ]:
photometry_results.sort('id')
plt.scatter(starlist['flux'], photometry_results['flux_fit'])
plt.xlabel('Ground-truth fluxes')
plt.ylabel('PSF Measured fluxes')
x = np.linspace(500, 1000)
plt.plot(x, x, color='orange');

Let's also plot the relative error on the fluxes estimation as a function of the ground-truth fluxes.

In [ ]:
rel_err = (photometry_results['flux_fit'] - starlist['flux']) / starlist['flux']
plt.scatter(starlist['flux'], rel_err)
plt.xlabel('Ground-truth flux')
plt.ylabel('Flux Relative Error')
plt.axhline(0.0, color='orange');

# `IterativelySubtractedPSFPhotometry`

`IterativelySubtractedPSFPhotometry` is a subclass of `BasicPSFPhotometry` that adds iteration functionality to the photometry procedure. It has the same parameters as `BasicPSFPhotometry`, except that it includes an additional `niters` parameter that represents the number of times to loop through the photometry process, subtracting the best-fit stars each time. The `niters` parameter can be `None`, which means that the photometry procedure will continue until no more sources are detected.

The  process implemented in `IterativelySubtractedPSFPhotometry` resembles the loop used by DAOPHOT: `FIND`, `GROUP`, `NSTAR`, `SUBTRACT`, `FIND`. On its own `IterativelySubtractedPSFPhotometry` doesn't implement the specific algorithms used in DAOPHOT (`DAOPhotPSFPhotometry`, discussed below, does), but it does implement the *structure* to enable this.

One final detail: the `finder` parameter (specifying the star-finder algorithm) for `IterativelySubtractedPSFPhotometry` cannot be `None` (as it can be for `BasicPSFPhotometry`). This is because it would not make sense to have an iterative process where the star finder changes completely at each step.  If you want to do that you're better off manually looping over a series of calls to different `BasicPSFPhotometry` objects.

## Example with unknown positions and unknown fluxes

Let's instantiate an object of `IterativelySubtractedPSFPhotometry`:

In [ ]:
from photutils.psf import IterativelySubtractedPSFPhotometry

itr_phot = IterativelySubtractedPSFPhotometry(group_maker=daogroup, bkg_estimator=mmm_bkg,
                                              psf_model=gaussian_psf, fitshape=fitshape,
                                              finder=iraffind, niters=2)

Let's now perform photometry on our artificial image:

In [ ]:
photometry_results = itr_phot(image)
photometry_results

Observe that there is a new column namely `iter_detected` which shows the iteration number in which the source was detected.

Let's plot the residual image.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 10))

im1 = ax1.imshow(image)
ax1.set_title('Simulated data')
plt.colorbar(orientation='horizontal', fraction=0.046, pad=0.04,
             ax=ax1, mappable=im1)

im2 = ax2.imshow(itr_phot.get_residual_image())
ax2.set_title('Residual Image')
plt.colorbar(orientation='horizontal', fraction=0.046, pad=0.04,
             ax=ax2, mappable=im2)

# `DAOPhotPSFPhotometry`

There is also a class called `DAOPhotPSFPhotometry` that is a subclass of `IterativelySubtractedPSFPhotometry`.  `DAOPhotPSFPhotometry` essentially implements the DAOPHOT photometry algorithm using `IterativelySubtractedPSFPhotometry`.  So instead of giving it arguments like `finder`, you provide parameters specific for the DAOPhot-like sub-tasks (e.g., the FWHM the star-finder is optimized for).

We leave the use of this class as an **exercise to the user** to play with the parameters which would optimize the photometry procedure. Uncomment the lines below and edit the cell:

In [ ]:
from photutils.psf import DAOPhotPSFPhotometry

#dao_phot = DAOPhotPSFPhotometry(...)

#photometry_results = dao_phot(image)
#photometry_results

## Documentation

Narrative and API docs of the classes used here can be found in https://photutils.readthedocs.io/en/stable/

# Future Work

The PSF Photometry module in photutils is still under development and feedback from users is much appreciated. Please open an issue on the github issue tracker of photutils with any suggestions for improvement, functionalities wanted, bugs, etc.

Near future implementations in the photutils.psf module include:

* FWHM estimation: a Python equivalent to DAOPHOT psfmeasure.